In [71]:
from experimentlog import ExperimentLog
import extract
import pseudo
import pandas as pd
# log some JSON data
e = ExperimentLog(":memory:", ntp_sync=False)
# now register the user with the database
user = pseudo.get_pseudo()
e.create("USER", name=user, data={"age":30, "leftright":"right"})
e.cd("/Experiment/Condition1")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0, "y":2})
e.cd("/Experiment/Condition2")
e.bind("USER",user)
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0})
e.leave()
e.enter('Condition2')
e.log("mouse", data={"x":0, "y":0})
e.log("mouse", data={"x":0, "y":1})
e.log("mouse", data={"x":0}, valid=0)
e.enter('SubconditionA')
e.end()


DEBUG:root:Opening database ':memory:'. Autocommit: 'None'
DEBUG:root:Creating tables.
DEBUG:root:Resuming from session 1 '/'
DEBUG:root:Run ID: [00000001]
DEBUG:root:Run config logged as '{}'
DEBUG:root:<Commit>
DEBUG:root:Registering 'GILAV-UTUKI' of type 'USER', with data [{"age": 30, "leftright": "right"}]
DEBUG:root:CD'ing to /Experiment/Condition1/
DEBUG:root:Parent session /
DEBUG:root:Entering session '/Experiment/'
DEBUG:root:New session ID [00000002]
DEBUG:root:<Commit>
DEBUG:root:Parent session /Experiment/
DEBUG:root:Entering session '/Experiment/Condition1/'
DEBUG:root:New session ID [00000003]
DEBUG:root:<Commit>
DEBUG:root:CD'd to /Experiment/Condition1/
DEBUG:root:Binding meta table USER:GILAV-UTUKI to /Experiment/Condition1/
02-16 16:30 [WARNI]  No stream mouse registered; creating a new blank entry
DEBUG:root:Registering 'mouse' of type 'STREAM', with data [null]
DEBUG:root:CD'ing to /Experiment/Condition2/
DEBUG:root:Leaving session '/Experiment/Condition1/'
DEBUG:ro

In [72]:
meta, bound_ix = extract.meta(e.cursor)
print extract.meta_dataframe(e.cursor)

{u'PATH':   bound  data description                                   name  type
0    []  None        None                           /Experiment/  None
1    []  None        None                /Experiment/Condition1/  None
2    []  None        None                /Experiment/Condition2/  None
3    []  None        None  /Experiment/Condition2/SubconditionA/  None, u'USER':     bound                                  data description         name type
0  [3, 4]  {u'age': 30, u'leftright': u'right'}              GILAV-UTUKI     , u'STREAM':   bound  data description   name  type
0    []  None              mouse  AUTO, u'DATASET':   bound                 data description  name  type
0    []  {u'stage': u'init'}        None  None  None}


In [73]:
print extract.dump_flat_dataframe(e.cursor)

{u'mouse':    session             t  x   y                     path  valid
0        3  1.455640e+09  0   0  /Experiment/Condition1/      1
1        3  1.455640e+09  0   1  /Experiment/Condition1/      1
2        3  1.455640e+09  0   2  /Experiment/Condition1/      1
3        4  1.455640e+09  0   0  /Experiment/Condition2/      1
4        4  1.455640e+09  0   1  /Experiment/Condition2/      1
5        4  1.455640e+09  0 NaN  /Experiment/Condition2/      1
6        5  1.455640e+09  0   0  /Experiment/Condition2/      1
7        5  1.455640e+09  0   1  /Experiment/Condition2/      1
8        5  1.455640e+09  0 NaN  /Experiment/Condition2/      0}


In [74]:
print extract.dump_dataframe(e.cursor)

defaultdict(<type 'list'>, {(u'/Experiment/Condition2/SubconditionA/',): [{}], (u'/Experiment/Condition1/',): [{u'mouse':    y  x  valid             t
0  0  0      1  1.455640e+09
1  1  0      1  1.455640e+09
2  2  0      1  1.455640e+09}], (u'/',): [{}], (u'/Experiment/',): [{}], (u'/Experiment/Condition2/',): [{u'mouse':     y  x  valid             t
0   0  0      1  1.455640e+09
1   1  0      1  1.455640e+09
2 NaN  0      1  1.455640e+09}, {u'mouse':     y  x  valid             t
0   0  0      1  1.455640e+09
1   1  0      1  1.455640e+09
2 NaN  0      0  1.455640e+09}]})


In [75]:
from collections import defaultdict
def session_tree(cursor):
    tree = defaultdict(list)
    parents = cursor.execute("SELECT id,parent FROM session").fetchall()
    for child, parent in parents:
        tree[parent].append(child)
    return tree

def expand_session(cursor):    
    full_tree = defaultdict(list)
    sessions = cursor.execute("SELECT id FROM session").fetchall()
    for session in sessions:
        s = session[0]
        orig = s
        parent = cursor.execute("SELECT parent FROM session WHERE id=?", (s,)).fetchone()[0]
        while parent!=None:
            full_tree[parent].append(orig)
            s = parent
            parent = cursor.execute("SELECT parent FROM session WHERE id=?", (s,)).fetchone()[0]
    return full_tree

def children(tree, node=None):
    ch = tree.get(node,[])    
    return ch + [children(tree, c) for c in ch if children(tree, c)!=[]]

def paths(cursor):
    paths = cursor.execute("SELECT path, count(id) FROM session GROUP BY path ORDER BY path").fetchall()    
    return paths
    
tree = session_tree(e.cursor)
print expand_session(e.cursor)
#print children(tree)
print "\n".join(["%-50s\tLogs:%d" % p for p in paths(e.cursor)])

defaultdict(<type 'list'>, {1: [2, 3, 4, 5, 6], 2: [3, 4, 5, 6], 5: [6]})
/                                                 	Logs:1
/Experiment/                                      	Logs:1
/Experiment/Condition1/                           	Logs:1
/Experiment/Condition2/                           	Logs:2
/Experiment/Condition2/SubconditionA/             	Logs:1
